In [1]:
import warnings
import pandas as pd
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('weekly data.csv')
df.drop('season', axis=1, inplace=True)
df['order_date'] = pd.to_datetime(df['order_date'])

In [3]:
frames = list()
for i in df['product_id'].unique():
    frame = df[df['product_id'] == i]
    frame = frame.sort_values(by='order_date')
    frames.append(frame)

df = pd.concat(frames, ignore_index=True)
df.head()

,order_date,product_id,quantities_sold
0,2019-09-01,8,4.0
1,2019-09-08,8,3.0
2,2019-09-15,8,0.0
3,2019-09-22,8,2.0
4,2019-09-29,8,2.0


In [6]:
pid = 8

frame = df[df['product_id'] == pid]
frame.drop('product_id', axis=1, inplace=True)
frame.rename(columns={'order_date':'ds', 'quantities_sold':'y'}, inplace=True)
frame_test = frame.tail(4)
frame = frame.drop(frame.tail(4).index)
frame.head()

,ds,y
0,2019-09-01,4.0
1,2019-09-08,3.0
2,2019-09-15,0.0
3,2019-09-22,2.0
4,2019-09-29,2.0


In [8]:
model = Prophet(weekly_seasonality=True, interval_width=0.98, seasonality_mode='additive')
model.fit(frame)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [16]:
dates = model.make_future_dataframe(periods=12,freq='W')  # 12 ==> number of weeks
len(dates)

129

In [17]:
forecast = model.predict(dates)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(12)


,ds,yhat,yhat_lower,yhat_upper
117,2021-11-28,1865.036994,1273.182125,2492.644843
118,2021-12-05,1883.295951,1224.368154,2527.249628
119,2021-12-12,1834.471845,1153.613694,2500.685542
120,2021-12-19,1807.617401,1156.893287,2422.658791
121,2021-12-26,1873.314259,1230.369342,2544.428954
122,2022-01-02,1998.142568,1391.393810,2641.568466
123,2022-01-09,2080.194159,1321.596276,2714.035568
124,2022-01-16,2064.429985,1409.749975,2673.213899
125,2022-01-23,2000.587979,1349.110176,2710.098543
126,2022-01-30,1977.562392,1296.129392,2624.069655


In [ ]:
plot_plotly(model, forecast)

In [ ]:
plot_components_plotly(model, forecast)
